

# Creating a Table from ![delta lake](https://delta.io/static/delta-lake-logo-a1c0d80d23c17de5f5d7224cb40f15dc.svg)

In this example notebook you will create a Table from data stored in the delta format.

## 📦 Install dependencies

Ensure you have the latest MLTable library and dependencies.

In [ ]:
%pip install -r ../mltable-requirements.txt

## 🐍 Create an MLTable using the Python SDK

Here you build your data loading steps using the `mltable` Python SDK. The `show()` method allows you to see the effect of the data loading transformation.

In [ ]:
import mltable
from mltable import DataType

tbl = mltable.from_delta_lake(
    delta_table_uri="wasbs://data@azuremlexampledata.blob.core.windows.net/COVID-19_NYT/",
    timestamp_as_of="2023-01-01T00:00:00Z",
)

# format the date column
data_types = {"date": DataType.to_datetime(formats=["%Y-%m-%d"])}

# convert the date column to a date type
tbl = tbl.convert_column_types(data_types)

# show the first 5 records
tbl.show(5)

### 🐼 Load into Pandas

You can load the data into pandas using:

In [ ]:
df = tbl.to_pandas_dataframe()
df.head()

### 💾 Save data loading steps 
Next, you'll save all your data loading steps into an `MLTable` file. This allows you to *reproduce* your Pandas data frame at a later point in time without having to redefine the data loading steps in your code.

In [ ]:
# save the data loading steps in an MLTable file
tbl.save("./covid")

#### 🔍 View the saved file

In the next code cell, we show you the `MLTable` file so you can understand how the data loading steps are serialized into a file.

In [ ]:
with open("./covid/MLTable", "r") as f:
    print(f.read())

## ♻️ Reproduce data loading steps

Now that the data loading steps have been serialized into a file, you can reproduce them at any point in time using the `load()` method. This means you do not need to redefine your data loading steps in code and makes it easier to share with others.

In [ ]:
import mltable

# load the previously saved MLTable file
tbl = mltable.load("./covid")
df = tbl.to_pandas_dataframe()
df.head(5)

### 🤝 Create a data asset to aid sharing and reproducibility

You'll now create a data asset, which will automatically upload the `MLTable` to cloud storage (the default AzureML datastore) so that others can use it easily.

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
import time
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# set the version number of the data asset to the current UTC time
VERSION = time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())

# connect to the AzureML workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

my_data = Data(
    path="./covid",
    type=AssetTypes.MLTABLE,
    description="COVID-19 cases in Washington county",
    name="covid-mltable-example",
    version=VERSION,
)

ml_client.data.create_or_update(my_data)

### 📖 Read the data asset in an interactive session

Now you have your MLTable stored in the cloud, you and Team members can access it using a friendly name in an interactive session (for example, a notebook).

In [ ]:
import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# connect to the AzureML workspace
# Note: the subscription, resource_group and workspace variables are set in a code cell above.
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# get the latest version of the data asset
# Note: the variable VERSION is created in a previous code cell
data_asset = ml_client.data.get(name="covid-mltable-example", version=VERSION)

# the table from the data asset id
tbl = mltable.load(f"azureml:/{data_asset.id}")

# load into pandas
df = tbl.to_pandas_dataframe()
df.head()

### 📖 Read the data asset in a job

You can also access your Table in a job, using:

In [ ]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential

# connect to the AzureML workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# get the latest version of the data asset
# Note: the VERSION was set in a previous cell.
data_asset = ml_client.data.get(name="covid-mltable-example", version=VERSION)

job = command(
    command="python train.py --input ${{inputs.covid}}",
    inputs={"covid": Input(type="mltable", path=data_asset.id)},
    compute="cpu-cluster",
    environment=Environment(
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        conda_file="./job-env/conda_dependencies.yml",
    ),
    code="./src",
)

ml_client.jobs.create_or_update(job)